# Importing libraries

In [1]:
!pip install keras-tuner --upgrade


In [5]:
# Importing Libraries
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras




#NEW
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras_tuner.tuners import BayesianOptimization
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns

# Importing datasets

In [7]:
# path to local data directory
data_dir = '/Users/air/Desktop/FinalProject/Data'

# Load data
X_rot_train = np.load(f'{data_dir}/X_rot_train.npy')
y_rot_train = np.load(f'{data_dir}/y_rot_train.npy')

X_rot_val = np.load(f'{data_dir}/X_rot_val.npy')
y_rot_val = np.load(f'{data_dir}/y_rot_val.npy')


# Creating a CNN and training it on rotations of image

In [ ]:

def build_model(hp):
    model = keras.models.Sequential([
        keras.layers.Conv2D(64, 7, activation="relu", padding="same",
                            input_shape=[32, 32, 3]),
        keras.layers.MaxPooling2D(2),
        keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
        keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
        keras.layers.MaxPooling2D(2),
        keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
        keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
        keras.layers.MaxPooling2D(2),
        keras.layers.Flatten(),
        keras.layers.Dense(128, activation="relu"),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(64, activation="relu"),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(4, activation="softmax")
    ])

    hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4])
    hp_epochs = hp.Int('epochs', min_value = 5, max_value = 30, step = 5)

    model.compile(optimizer = keras.optimizers.SGD(learning_rate=hp_learning_rate),
                  loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics = ['accuracy'])

    return model



# Run Bayesian Optimization

In [ ]:
# Run Bayesian Optimization
tuner = BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=1,
    directory='/content/drive/MyDrive/A Goldsmiths /A Final Project/DEV/AutoBayesOptRot50epoch',
    project_name='RotationPred',
    overwrite=True #True in the tuner will overwrite, to continue from a previous state, set False
)
tuner.search_space_summary()
tuner.search(X_rot_train, y_rot_train, epochs=50, validation_data=(X_rot_val, y_rot_val))


# Get the optimal hyperparameters
opt_hps = tuner.get_best_hyperparameters(num_trials=1)[0]


In [ ]:
# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(opt_hps)


In [ ]:
# Checking the Model Architecture
model.summary()


In [ ]:
#SAVE MODEL
from google.colab import drive
drive.mount('/content/gdrive')
model_name = "Rot-PretextBayesian50epoch"
path = f"/content/gdrive/MyDrive/A Goldsmiths /A Final Project/DEV/{model_name}"
model.save(path)

#New Metrics

In [ ]:
# Compute metrics and create plots
predicted_labels = np.argmax(model.predict(X_rot_val), axis=1)

# Compute accuracy
accuracy = accuracy_score(y_rot_val, predicted_labels)
print('Accuracy: ', accuracy)

# Compute confusion matrix
cm = confusion_matrix(y_rot_val, predicted_labels)
print('Confusion Matrix: ')
print(cm)


In [ ]:
# Plot confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:

# Precision, Recall, F1-score with 'macro' average
precision_macro = precision_score(y_rot_val, predicted_labels, average='macro')
recall_macro = recall_score(y_rot_val, predicted_labels, average='macro')
f1_macro = f1_score(y_rot_val, predicted_labels, average='macro')

# Precision, Recall, F1-score with 'micro' average
precision_micro = precision_score(y_rot_val, predicted_labels, average='micro')
recall_micro = recall_score(y_rot_val, predicted_labels, average='micro')
f1_micro = f1_score(y_rot_val, predicted_labels, average='micro')

# Create a DataFrame for the results
results = pd.DataFrame({
    'Metric': ['Macro Precision', 'Macro Recall', 'Macro F1-score', 'Micro Precision', 'Micro Recall', 'Micro F1-score'],
    'Score': [precision_macro, recall_macro, f1_macro, precision_micro, recall_micro, f1_micro]
})

print(results)

##Unsupervised Metrics

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, classification_report

# Predict the rotation of the validation set
y_rot_val_pred = model.predict(X_rot_val).argmax(axis=1)  # Returns the indices of the maximum values along an axis

# Accuracy
accuracy = accuracy_score(y_rot_val, y_rot_val_pred)
print(f"Accuracy: {accuracy}")

# Confusion Matrix
conf_matrix = confusion_matrix(y_rot_val, y_rot_val_pred)
print(f"Confusion Matrix:\n {conf_matrix}")

# Precision, Recall, F1-score with 'macro' average
precision_macro = precision_score(y_rot_val, y_rot_val_pred, average='macro')
recall_macro = recall_score(y_rot_val, y_rot_val_pred, average='macro')
f1_macro = f1_score(y_rot_val, y_rot_val_pred, average='macro')
print(f"Macro Precision: {precision_macro}")
print(f"Macro Recall: {recall_macro}")
print(f"Macro F1-score: {f1_macro}")

# Precision, Recall, F1-score with 'micro' average
precision_micro = precision_score(y_rot_val, y_rot_val_pred, average='micro')
recall_micro = recall_score(y_rot_val, y_rot_val_pred, average='micro')
f1_micro = f1_score(y_rot_val, y_rot_val_pred, average='micro')
print(f"Micro Precision: {precision_micro}")
print(f"Micro Recall: {recall_micro}")
print(f"Micro F1-score: {f1_micro}")

# Classification Report
print("Classification Report:\n", classification_report(y_rot_val, y_rot_val_pred))


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, classification_report

# Predict the rotation of the validation set
y_rot_val_pred = model.predict(X_rot_val).argmax(axis=1)

# Accuracy
accuracy = accuracy_score(y_rot_val, y_rot_val_pred)
print(f"Accuracy: {accuracy}\n")

# Confusion Matrix
conf_matrix = confusion_matrix(y_rot_val, y_rot_val_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.title('Confusion Matrix')
plt.show()

# Precision, Recall, F1-score with 'macro' average
precision_macro = precision_score(y_rot_val, y_rot_val_pred, average='macro')
recall_macro = recall_score(y_rot_val, y_rot_val_pred, average='macro')
f1_macro = f1_score(y_rot_val, y_rot_val_pred, average='macro')

# Precision, Recall, F1-score with 'micro' average
precision_micro = precision_score(y_rot_val, y_rot_val_pred, average='micro')
recall_micro = recall_score(y_rot_val, y_rot_val_pred, average='micro')
f1_micro = f1_score(y_rot_val, y_rot_val_pred, average='micro')

# Create a DataFrame for the results
results = pd.DataFrame({
    'Metric': ['Macro Precision', 'Macro Recall', 'Macro F1-score', 'Micro Precision', 'Micro Recall', 'Micro F1-score'],
    'Score': [precision_macro, recall_macro, f1_macro, precision_micro, recall_micro, f1_micro]
})

print(results)


In [ ]:
#SAVE MODEL
from google.colab import drive
drive.mount('/content/gdrive')
model_name = "Unsupervised"
path = f"/content/gdrive/MyDrive/A Goldsmiths /A Final Project/DEV/{model_name}"
model.save(path)

In [ ]:
from google.colab import files

model.save('Rot-PretextTask.h5')

files.download('Rot-PretextTask.h5')

#Supervised Model

In [ ]:
# Removing the top layer and addding a new top layer
model.pop()
model.add(keras.layers.Dense(10, name='dense_3', activation='softmax'))

In [ ]:


# Checking the changes in Model Architecture
model.summary()

In [ ]:
# Check the 'Trainable' status of each layer
for layer in model.layers:
    print(layer.name, layer.trainable)

In [ ]:
# Freezing the Convolutional Layers while keeping Dense layers as Trainable
for layer in model.layers:
    if layer.name in ['dense_3', 'dense_1', 'dense', 'dropout', 'dropout_1']:
      layer.trainable=True
    else:
      layer.trainable=False

In [ ]:
# Checking if the changes in 'Trainable' status of each layer have taken place
for layer in model.layers:
    print(layer.name, layer.trainable)

#DATA BITS START

In [ ]:
# Reshaping the Inputs
X_labeled=X_labeled.reshape(-1, 32, 32, 3)
X_val=X_val.reshape(-1, 32, 32, 3)
X_test=X_test.reshape(-1, 32, 32, 3)

In [ ]:
# Compiling the model
model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])

# Training the model on Downstream Task
history=model.fit(X_labeled, y_labeled, validation_data=(X_val, y_val), epochs=10)

In [ ]:
#SAVE Superv MODEL
#Source: https://machinelearningmastery.com/save-load-keras-deep-learning-models/#:~:text=The%20weights%20are%20saved%20directly,the%20symmetrical%20load_weights()%20function.
from google.colab import drive
drive.mount('/content/gdrive')
model_name = "supervised.h5"
path = f"/content/gdrive/My Drive/A Final Project/Dev/{model_name}"
model.save(path)


In [ ]:
# Evaluating the model on the Test set
model.evaluate(X_val, y_val)

In [ ]:
from google.colab import files

model.save('TargetModel.h5')

files.download('TargetModel.h5')

#Supervised Learning Metrics


In [ ]:
import matplotlib.pyplot as plt

# Load data
#X_test = np.load('/content/drive/MyDrive/A Goldsmiths /A Final Project/DEV/Data/X_test.npy')
#y_test = np.load('/content/drive/MyDrive/A Goldsmiths /A Final Project/DEV/Data/y_test.npy')

# Print a slice of each array
print("X_val:\n", X_val[:5])
print("\ny_val:\n", y_val[:5])

# Training and evaluation

eval_result = model.evaluate(X_val, y_val)
print("\nTest loss, Test accuracy:", eval_result)

# Visualizing the metrics
fig, axs = plt.subplots(2)

# summarize history for accuracy
axs[0].plot(history.history['accuracy'])
axs[0].plot(history.history['val_accuracy'])
axs[0].set_title('Model accuracy')
axs[0].set_ylabel('Accuracy')
axs[0].set_xlabel('Epoch')
axs[0].legend(['train', 'validation'], loc='upper left')

# summarize history for loss
axs[1].plot(history.history['loss'])
axs[1].plot(history.history['val_loss'])
axs[1].set_title('Model loss')
axs[1].set_ylabel('Loss')
axs[1].set_xlabel('Epoch')
axs[1].legend(['train', 'validation'], loc='upper left')

plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import (accuracy_score, confusion_matrix, precision_score, recall_score, f1_score,
                             classification_report, log_loss, roc_auc_score, roc_curve, auc)
from sklearn.preprocessing import label_binarize
import numpy as np

# Predict the classes and probabilities of the validation set
y_val_pred = model.predict(X_val).argmax(axis=1)
y_val_prob = model.predict(X_val)  # assuming predict gives probabilities

# Accuracy
accuracy = accuracy_score(y_val, y_val_pred)
print(f"Accuracy: {accuracy}\n")

# Confusion Matrix
conf_matrix = confusion_matrix(y_val, y_val_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.title('Confusion Matrix')
plt.show()

# Precision, Recall, F1-score with 'macro' average
precision_macro = precision_score(y_val, y_val_pred, average='macro')
recall_macro = recall_score(y_val, y_val_pred, average='macro')
f1_macro = f1_score(y_val, y_val_pred, average='macro')

# Precision, Recall, F1-score with 'micro' average
precision_micro = precision_score(y_val, y_val_pred, average='micro')
recall_micro = recall_score(y_val, y_val_pred, average='micro')
f1_micro = f1_score(y_val, y_val_pred, average='micro')

# Log Loss
logloss = log_loss(y_val, y_val_prob)

# Create a DataFrame for the results
results = pd.DataFrame({
    'Metric': ['Accuracy', 'Macro Precision', 'Macro Recall', 'Macro F1-score',
               'Micro Precision', 'Micro Recall', 'Micro F1-score', 'Log Loss'],
    'Score': [accuracy, precision_macro, recall_macro, f1_macro,
              precision_micro, recall_micro, f1_micro, logloss]
})

print(results)

# ROC and AUC
n_classes = 10

# Binarize the output
y_val_bin = label_binarize(y_val, classes=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_val_bin[:, i], y_val_prob[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot ROC curve
for i in range(n_classes):
    plt.figure()
    plt.plot(fpr[i], tpr[i], label='ROC curve (area = %0.2f)' % roc_auc[i])
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    plt.show()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import (accuracy_score, confusion_matrix, precision_score, recall_score, f1_score,
                             classification_report, log_loss, roc_auc_score, roc_curve, auc)
from sklearn.preprocessing import label_binarize
import numpy as np

# CIFAR-10 class names
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# Predict the classes and probabilities of the validation set
y_val_pred = model.predict(X_val).argmax(axis=1)
y_val_prob = model.predict(X_val)  # assuming predict gives probabilities

# Accuracy
accuracy = accuracy_score(y_val, y_val_pred)
print(f"Accuracy: {accuracy}\n")

# Confusion Matrix
conf_matrix = confusion_matrix(y_val, y_val_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.title('Confusion Matrix')
plt.show()

# Precision, Recall, F1-score with 'macro' average
precision_macro = precision_score(y_val, y_val_pred, average='macro')
recall_macro = recall_score(y_val, y_val_pred, average='macro')
f1_macro = f1_score(y_val, y_val_pred, average='macro')

# Precision, Recall, F1-score with 'micro' average
precision_micro = precision_score(y_val, y_val_pred, average='micro')
recall_micro = recall_score(y_val, y_val_pred, average='micro')
f1_micro = f1_score(y_val, y_val_pred, average='micro')

# Log Loss
logloss = log_loss(y_val, y_val_prob)

# Create a DataFrame for the results
results = pd.DataFrame({
    'Metric': ['Accuracy', 'Macro Precision', 'Macro Recall', 'Macro F1-score',
               'Micro Precision', 'Micro Recall', 'Micro F1-score', 'Log Loss'],
    'Score': [accuracy, precision_macro, recall_macro, f1_macro,
              precision_micro, recall_micro, f1_micro, logloss]
})

print(results)

# ROC and AUC
n_classes = 10

# Binarize the output
y_val_bin = label_binarize(y_val, classes=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_val_bin[:, i], y_val_prob[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot ROC curve
for i in range(n_classes):
    plt.figure()
    plt.plot(fpr[i], tpr[i], label='ROC curve (area = %0.2f) for %s' % (roc_auc[i], class_names[i]))
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    plt.show()


##Print

In [ ]:
!jupyter nbconvert --to html /content/SSL_model.ipynb